In [18]:
pip install yesg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=2d2e55977eb68b65b23ae91c68d37f52c5afb1a11bfd6ffe647bfec6dc71bbaa
  Stored in directory: /Users/johnkim/Library/Caches/pip/wheels/78/8d/48/f5e8ff0315a46301e15c68371e297b460b33e1c846117725bc
Successfully built yesg
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import yfinance as yf
import time
from random import randint
import yesg
import requests

In [2]:
# There are 2 tables on the Wikipedia page, we want the first table
# second table shows the evolution of the changes

components = \
pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')

first_table = components[0]
second_table = components[1]
sp_df = first_table
sp_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [11]:
sp500 = sp_df['Symbol']

In [3]:
url = 'https://query2.finance.yahoo.com/v1/finance/esgChart'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0'}

# List of dataframes
dfs = {}

for symbol in sp_df['Symbol']:
    response = requests.get(url, headers=headers, params={'symbol': symbol})
    data = response.json()
    if response.ok:
        # skip symbol if has no ESG data
        try:
            df = pd.DataFrame(data['esgChart']['result'][0]['symbolSeries'])
            dfs[symbol] = df
        except KeyError:
            pass            

df = pd.concat(dfs, names=['symbol']).reset_index(level='symbol')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [4]:
df

,symbol,timestamp,esgScore,governanceScore,environmentScore,socialScore
0,MMM,2014-09-01,73.00,73.00,73.00,74.00
1,MMM,2014-10-01,72.00,73.00,72.00,74.00
2,MMM,2014-11-01,73.00,73.00,73.00,74.00
3,MMM,2014-12-01,73.00,73.00,73.00,74.00
4,MMM,2015-01-01,74.00,78.00,70.00,76.00
...,...,...,...,...,...,...
104,ZTS,2023-05-01,NaN,NaN,NaN,NaN
105,ZTS,2023-06-01,NaN,NaN,NaN,NaN
106,ZTS,2023-07-01,NaN,NaN,NaN,NaN
107,ZTS,2023-08-01,NaN,NaN,NaN,NaN


In [10]:
# All available historic ESG rating for THE BOEING COMPANY
print(yesg.get_historic_esg('BA'))

            Total-Score  E-Score  S-Score  G-Score
Date                                              
2014-09-01         60.0    64.00    51.00    64.00
2014-10-01         60.0    64.00    51.00    64.00
2014-11-01         60.0    64.00    51.00    64.00
2014-12-01         60.0    64.00    51.00    64.00
2015-01-01         62.0    69.00    54.00    64.00
...                 ...      ...      ...      ...
2023-05-01          NaN      NaN      NaN      NaN
2023-06-01          NaN      NaN      NaN      NaN
2023-07-01          NaN      NaN      NaN      NaN
2023-08-01          NaN      NaN      NaN      NaN
2023-09-01         39.6     8.84    22.48     8.29

[109 rows x 4 columns]
